In [1]:
# Import all functions from the required modules
from cordo_sherpa_module import *
from plot_module import *
#from association_module import *
print("Successfully loaded all modules")

Successfully loaded plotting command
Successfully loaded all modules


In [2]:
# Paths to the files
path_fichier_shp = "data/2-output-data/donnees_shp"
title_shp = "donnees_paris_donnees_insee_iris"
path_fichier_pourcents = "data/2-output-data"
title_pourcents = "pourcents"

# Load the concentration points
conc_points = coordo_sherpa(sc="s1", pol="ug_NO2", year=2019)

# Load the exported data
donnees_exportees = gpd.read_file(os.path.join(path_fichier_shp, f"{title_shp}.shp"))

# Transform the CRS of the exported data to match the concentration points
donnees_exportees_transformed = donnees_exportees.to_crs(conc_points.crs)

# Check if CRSs are the same
if conc_points.crs == donnees_exportees_transformed.crs:
    print("CRS for conc_points_transformed and donnees_exportees_transformed are the same.")
else:
    print("CRS for conc_points_transformed and donnees_exportees_transformed are different.")

Concentrations in 2019 and 2019 are calculated for the pollutant 'ug_no2' (s1).
CRS for conc_points_transformed and donnees_exportees_transformed are the same.


In [3]:
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.ops import unary_union
import pandas as pd
import numpy as np
from functools import partial


def generate_points(polygone, conc_points):
    """
    Generate grid points for a given polygon and associate them with concentration points.
    """
    if polygone.geometry.is_empty or not polygone.geometry.is_valid:
        return gpd.GeoDataFrame()  # Return empty GeoDataFrame if invalid

    if conc_points is None or conc_points.empty:
        return gpd.GeoDataFrame()  # Return empty GeoDataFrame if conc_points is empty

    # Filter concentration points within the polygon
    points_within = conc_points[conc_points.geometry.within(polygone.geometry)].copy()
    points_within['iriscod'] = polygone['iriscod']

    return points_within

def calculate_intersection_percentages(grille_combinee, donnees_exportees_transformed):
    """
    Iteratively calculate the intersection percentages for each grid point's rectangle and associated polygon.
    Returns:
        - GeoDataFrame: grille_combinee with calculated 'perc' values.
    """
    # Initialize 'perc' column with zeros
    grille_combinee['perc'] = 0

    # Iterate through each row in grille_combinee
    for idx, row in grille_combinee.iterrows():
        # Extract coordinates of the point
        x, y = row.geometry.x, row.geometry.y

        # Create a rectangle (polygon) around the point
        rectangle = Polygon([
            (round(x - 0.05, 3), round(y - 0.025, 3)),
            (round(x + 0.05, 3), round(y - 0.025, 3)),
            (round(x + 0.05, 3), round(y + 0.025, 3)),
            (round(x - 0.05, 3), round(y + 0.025, 3)),
            (round(x - 0.05, 3), round(y - 0.025, 3))
        ])

        # Filter the polygon corresponding to the current grid point's 'iriscod'
        iriscod = row['iriscod']
        polygon_row = donnees_exportees_transformed[donnees_exportees_transformed['iriscod'] == iriscod]

        if polygon_row.empty or polygon_row.geometry.is_empty.any():
            continue  # Skip if no matching polygon or geometry is invalid

        # Extract the polygon geometry
        polygon_geom = polygon_row.geometry.iloc[0]

        # Intersection between the rectangle and the polygon
        intersection = rectangle.intersection(polygon_geom)

        if not intersection.is_empty:
            area_intersection = intersection.area
            area_polygon = polygon_geom.area

            # Calculate the percentage of intersection
            perc = round(area_intersection / area_polygon, 3)
            grille_combinee.at[idx, 'perc'] = perc

    # Filter rows where 'perc' is greater than 0
    grille_combinee = grille_combinee[grille_combinee['perc'] > 0]

    return grille_combinee

from geopandas import GeoDataFrame  # Ensure correct imports
def process_subset(subset, conc_points):
    # Make sure subset and conc_points are correct types
    if not isinstance(subset, GeoDataFrame):
        raise TypeError(f"Expected 'subset' to be a GeoDataFrame, got {type(subset)}")
    if not isinstance(conc_points, GeoDataFrame):
        raise TypeError(f"Expected 'conc_points' to be a GeoDataFrame, got {type(conc_points)}")

    # Perform operations on the 'subset' that involve iterating over rows
    # Here, I'm assuming 'conc_points' needs to be used within the process
    result_rows = []
    for idx, row in subset.iterrows():
        # Perform some operation with `row` and `conc_points`
        result_rows.append(row)  # This is just a placeholder for actual logic

    # Convert the result back to a GeoDataFrame
    return GeoDataFrame(result_rows)



def worker_function(donnees_subset, conc_points):
    """
    Wrapper to process subsets of polygons using multiprocessing.
    """
    try:
        return process_subset(donnees_subset, conc_points)
    except Exception as e:
        print(f"Worker function error: {e}")
        return gpd.GeoDataFrame()

In [8]:
from geopandas import GeoDataFrame
import pandas as pd
from multiprocessing import Pool


def split_into_subsets(data, num_splits=4):
    """
    Splits a GeoDataFrame into subsets for multiprocessing.
    Args:
        data (GeoDataFrame): The GeoDataFrame to be split.
        num_splits (int): The number of subsets to create.
    Returns:
        List[GeoDataFrame]: A list of GeoDataFrame subsets.
    """
    if len(data) == 0:
        return []  # Return an empty list if the input data is empty

    # Split the data into evenly sized chunks
    chunk_size = max(1, len(data) // num_splits)
    return [data.iloc[i:i + chunk_size] for i in range(0, len(data), chunk_size)]


def worker_func(subset, conc_points):
    """
    Worker function to process a subset of data.
    """
    # Add error-handling or debug statements inside worker_func
    if subset.empty:
        return GeoDataFrame()  # Return an empty GeoDataFrame for empty input

    # Some complex processing logic here which returns a GeoDataFrame
    # Example: spatial join, buffering, intersection, etc.
    try:
        processed_result = some_spatial_operations(subset, conc_points)
        return processed_result
    except Exception as e:
        # Log the error for debugging purposes
        print(f"Error processing subset: {e}")
        return GeoDataFrame()  # Ensure the function always returns a GeoDataFrame


def association(donnees_exportees_transformed, conc_points):
    """
    This function performs a spatial association between two geospatial datasets.
    """
    # Split the data into subsets for multiprocessing
    subsets = split_into_subsets(donnees_exportees_transformed)

    if not subsets:  # Check if subsets is empty before proceeding
        raise ValueError("The input data resulted in empty subsets. Please check the input.")

    # Use multiprocessing only if subsets are valid
    with Pool() as pool:
        # Pass conc_points as an additional argument to worker_func
        grids = pool.starmap(worker_func, [(subset, conc_points) for subset in subsets])

    # Remove empty results from grids before concatenating
    non_empty_grids = [grid for grid in grids if not grid.empty]

    if not non_empty_grids:
        # Raise error if there's no valid data after processing
        raise ValueError("All subsets returned empty results. Check worker_func and input data.")

    # Concatenate the results into a single GeoDataFrame
    result = GeoDataFrame(pd.concat(non_empty_grids, ignore_index=True))

    return result

In [ ]:
import matplotlib.pyplot as plt

# GLOBAL FUNCTION FOR MULTIPROCESSING
def worker_function(subset, conc_points):
    return process_subset(subset, conc_points)


if __name__ == '__main__':
    # Paths and parameters
    path_fichier_shp = "data/2-output-data/donnees_shp"
    title_shp = "donnees_insee_iris"
    path_fichier_pourcents = "data/2-output-data"
    title_pourcents = "pourcents_shp"

    # Step 1: Load exported shapefile data
    donnees_exportees_transformed = gpd.read_file(os.path.join(path_fichier_shp, f"{title_shp}.shp"))

    # Step 2: Load concentration points
    conc_points = coordo_sherpa(sc="s1", pol="ug_NO2", year=2019)

    # Step 3: Ensure CRS compatibility
    donnees_exportees_transformed = donnees_exportees_transformed.to_crs(conc_points.crs)

    # Check CRS validity
    if donnees_exportees_transformed.crs != conc_points.crs:
        print("CRS mismatch detected; converting CRS...")
        conc_points = conc_points.to_crs(donnees_exportees_transformed.crs)
    else:
        print("CRS for conc_points and donnees_exportees_transformed are the same.")

    # Step 4: Perform association
    donnees_pourcents = association(donnees_exportees_transformed, conc_points)

    # Step 5: Export shapefile
    path_result = os.path.join(path_fichier_pourcents, f"{title_pourcents}.shp")
    donnees_pourcents.to_file(path_result, driver='ESRI Shapefile')

    print(f"Results saved to {path_result}")

    # Step 6 (Optional): Visualization
    try:
        donnees_pourcents.plot(column='perc', cmap='viridis', legend=True)
        plt.title("Intersection Percentages")
        plt.show()
    except Exception as e:
        print(f"An error occurred during plotting: {e}")


Concentrations in 2019 and 2019 are calculated for the pollutant 'ug_no2' (s1).
CRS for conc_points and donnees_exportees_transformed are the same.
